In [297]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import pycountry
from collections import defaultdict
import requests


# Load Verra data 

In [2]:
r = requests.post(
    'https://registry.verra.org/uiapi/asset/asset/search?$maxResults=2000&$count=true&$skip=0&format=csv',
    json={"program":"VCS","issuanceTypeCodes":['ISSUE']}
)
rename_map = {
 'issuanceDate':'Issuance Date', 
 'programObjectives': 'Sustainable Development Goals', 
 'instrumentType': 'Credit Type', 
 'vintageStart':'Vintage Start',
 'vintageEnd':'Vintage End', 
 'reportingPeriodStart':'Reporting Period Start', 
 'reportingPeriodEnd':'Reporting Period End',
 'resourceIdentifier':'ID', 
 'resourceName':'Name', 
 'region':'Region', 
 'country':'Country',
 'protocolCategory': 'Project Type', 
 'protocol':'Methodology', 
 'totalVintageQuantity':'Total Vintage Quantity', 
 'quantity':'Quantity Issued',
 'serialNumbers':'Serial Number', 
 'additionalCertifications': 'Additional Certifications', 
 'retiredCancelled':'Is Cancelled',
 'retireOrCancelDate':'Retirement/Cancellation Date', 
 'retirementBeneficiary':'Retirement Beneficiary', 
 'retirementReason':'Retirement Reason',
 'retirementDetails':'Retirement Details', 
 'inputTypes': 'Input Type', 
 'holdingIdentifier': 'Holding ID'
}
vcus = pd.DataFrame(r.json()['value']).rename(columns=rename_map)

# Fixing up datetimes
vcus['Vintage Start'] = pd.to_datetime(vcus['Vintage Start'])
vcus['Vintage End'] = pd.to_datetime(vcus['Vintage End'])
vcus['Issuance Date'] = pd.to_datetime(vcus['Issuance Date'])
vcus['Retirement/Cancellation Date'] = pd.to_datetime(vcus['Retirement/Cancellation Date'])
# Assgining vintage year
vcus['Vintage'] = vcus['Vintage Start'].dt.year
vcus['Issuance Year'] = vcus['Issuance Date'].dt.year
# Assigning retirement age
vcus['Days to Retirement'] = (vcus['Retirement/Cancellation Date'] - vcus['Issuance Date']).dt.days
# Assigning staleness if not retired
now = dt.datetime.now()
vcus.loc[vcus['Days to Retirement'].isna(),'Staleness'] = (now-vcus[vcus['Days to Retirement'].isna()]['Issuance Date']).dt.days
# Assigning retirement status 
vcus['Status'] = ['Retired' if row['Days to Retirement']>0 else 'Available' for i, row in vcus.iterrows()]
# Some nice renaming since these are too verbose
vcus = vcus.rename(columns={'Total Vintage Quantity': 'Vintage Quantity', 'Quantity Issued': 'Quantity'})
# Toucan retirement credits
vcus.loc[vcus['Retirement Details'].str.contains('TOUCAN').fillna(False), 'Toucan'] = True
vcus['Toucan'] = vcus['Toucan'].fillna(False)
vcus.loc[vcus['Toucan'], 'Toucan Bridger'] = "https://polygonscan.com/address/"+vcus.query('Toucan')['Retirement Beneficiary']
# Toucan AM0001 methodology blacklist
toucan_blacklist_date = dt.datetime(year=2021, month=12, day=13)
vcus['Toucan Blacklisted'] = False
vcus.loc[np.logical_and(vcus['Retirement/Cancellation Date']>=toucan_blacklist_date, vcus['Methodology']=="AM0001"),'Toucan Blacklisted'] = True
# Building up an index for country codes
vcus['Country'] = vcus['Country'].replace('South Korea', 'Korea, Republic of')
country_index = defaultdict(str,{country:pycountry.countries.search_fuzzy(country)[0].alpha_3 for country in vcus.Country.astype(str).unique() if country!='nan'})
# ... so we could have country codes for visualizations
vcus['Country Code'] = [country_index[country] for country in vcus['Country']]
# Are vintage batches ever larger than 1 year?
vcus['Vintaged Batch Years'] = (vcus['Vintage End']-vcus['Vintage Start']).dt.days/365

# Relevant day stuff
last_time = dt.datetime.combine(dt.date.today(), dt.datetime.min.time())
seven_day_start = last_time - dt.timedelta(days=7)
last_seven_day_start = seven_day_start - dt.timedelta(days=7)
thirty_day_start = last_time - dt.timedelta(days=30)
last_thirty_day_start = thirty_day_start - dt.timedelta(days=30)
tpool = vcus.query('Toucan and ~`Toucan Blacklisted`')

# Seven day pool subsets    
sd_pool = tpool.query("(`Retirement/Cancellation Date`>=@seven_day_start) and (`Retirement/Cancellation Date`<@last_time)")
last_sd_pool = tpool.query("(`Retirement/Cancellation Date`>=@last_seven_day_start) and (`Retirement/Cancellation Date`<@seven_day_start)")
# Thirty day pool subsets
td_pool = tpool.query("(`Retirement/Cancellation Date`>=@thirty_day_start) and (`Retirement/Cancellation Date`<@last_time)")
last_td_pool = tpool.query("(`Retirement/Cancellation Date`>=@last_thirty_day_start) and (`Retirement/Cancellation Date`<@thirty_day_start)")

In [354]:
vcus['ID'] = vcus['ID'].astype(int)

In [355]:
vcus.columns

Index(['Issuance Date', 'Sustainable Development Goals', 'Credit Type',
       'Vintage Start', 'Vintage End', 'Reporting Period Start',
       'Reporting Period End', 'ID', 'Name', 'Region', 'Country',
       'Project Type', 'Methodology', 'Vintage Quantity', 'Quantity',
       'Serial Number', 'Additional Certifications', 'Is Cancelled',
       'Retirement/Cancellation Date', 'Retirement Beneficiary',
       'Retirement Reason', 'Retirement Details', 'Input Type', 'Holding ID',
       'Vintage', 'Issuance Year', 'Days to Retirement', 'Staleness', 'Status',
       'Toucan', 'Toucan Bridger', 'Toucan Blacklisted', 'Country Code',
       'Vintaged Batch Years', 'has_ac'],
      dtype='object')

In [356]:
vcus.groupby('Project Type')['Project Type'].count()

Project Type
Agriculture Forestry and Other Land Use                                                                                      98029
Agriculture Forestry and Other Land Use; Energy industries (renewable/non-renewable sources); Waste handling and disposal        1
Chemical industry                                                                                                              749
Chemical industry; Energy industries (renewable/non-renewable sources)                                                          92
Construction; Energy industries (renewable/non-renewable sources)                                                              250
Energy demand                                                                                                                  897
Energy demand; Energy industries (renewable/non-renewable sources)                                                             136
Energy demand; Waste handling and disposal                            

# Additional Certifications flag

In [357]:
vcus['Additional Certifications'].value_counts()

CCB-Gold                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              47026
Social Carbon                                                                                                                                                                                                                                                                                                                                                                                                                               

In [358]:
vcus['has_ac'] = ~vcus['Additional Certifications'].isnull()

In [359]:
vcus.groupby('has_ac')['Quantity'].sum()

has_ac
False    595439360
True     311556281
Name: Quantity, dtype: int64

# Label retired vs. unretired credits

In [360]:
vcus['Retirement/Cancellation Date'].isnull().value_counts()

False    146804
True      13088
Name: Retirement/Cancellation Date, dtype: int64

In [361]:
rets = vcus[~vcus['Retirement/Cancellation Date'].isnull()]

In [362]:
rets['Retirement/Cancellation Date'].isnull().value_counts()

False    146804
Name: Retirement/Cancellation Date, dtype: int64

# Label tokenized credits

In [363]:
rets = rets[~rets['Retirement Details'].str.contains('TOUCAN|C3T').fillna(False)]

# Define Demand

### Features
- Categories (+ ACs)
- Delta btwn issuance and retirement
- Delta btwn issuance and vintage
- Delta btwn vintage and retirement
- % of blank retirements
- % of issued credits retired (by project)

Compare % of issued vs. retired against baseline avg for all of Verra

In [364]:
rets['issue_retire_delta'] = rets['Retirement/Cancellation Date'] - rets['Issuance Date']

In [365]:
rets['vintage_retire_delta'] = rets['Retirement/Cancellation Date'] - rets['Vintage Start']

In [366]:
rets['vintage_issuance_delta'] = rets['Issuance Date'] - rets['Vintage Start']

In [367]:
rets['issue_retire_delta'].value_counts()

71 days      1329
1 days       1171
320 days      940
70 days       917
72 days       890
             ... 
3842 days       1
3766 days       1
2308 days       1
3716 days       1
2883 days       1
Name: issue_retire_delta, Length: 3534, dtype: int64

In [368]:
rets['vintage_retire_delta'].value_counts()

1206 days    2132
1207 days    1748
1208 days    1642
2301 days    1090
1209 days     791
             ... 
6057 days       1
6064 days       1
6067 days       1
6070 days       1
5087 days       1
Name: vintage_retire_delta, Length: 5539, dtype: int64

In [369]:
rets['vintage_issuance_delta'].value_counts()

2303 days    5126
1923 days    3577
3644 days    3525
1129 days    3415
1135 days    2947
             ... 
2847 days       1
4553 days       1
3456 days       1
2843 days       1
145 days        1
Name: vintage_issuance_delta, Length: 3268, dtype: int64

In [370]:
rets['is_blank'] = rets['Retirement Beneficiary'].isnull()

In [371]:
rets['is_blank'].value_counts()

False    73270
True     72654
Name: is_blank, dtype: int64

### Join Mapping Data

In [372]:
mapping = pd.read_excel('./MAPPING_subset.xlsx')

In [373]:
mapping.columns

Index(['ID', 'technology', 'category'], dtype='object')

In [374]:
mapping['ID'] = mapping.ID.astype(int)

In [375]:
mapping = mapping[['ID', 'technology', 'category']]

In [376]:
mapping.head()

,ID,technology,category
0,2592,LFG,EE
1,2588,WIND,RE
2,2557,Water purification,WATER PURIFICATION
3,2537,WASTE,EE
4,2533,COOKSTOVES,EE SMALL SCALE


In [377]:
mapping.dtypes

ID             int64
technology    object
category      object
dtype: object

In [378]:
rets = pd.merge(rets, mapping, on='ID')

In [379]:
rets.columns

Index(['Issuance Date', 'Sustainable Development Goals', 'Credit Type',
       'Vintage Start', 'Vintage End', 'Reporting Period Start',
       'Reporting Period End', 'ID', 'Name', 'Region', 'Country',
       'Project Type', 'Methodology', 'Vintage Quantity', 'Quantity',
       'Serial Number', 'Additional Certifications', 'Is Cancelled',
       'Retirement/Cancellation Date', 'Retirement Beneficiary',
       'Retirement Reason', 'Retirement Details', 'Input Type', 'Holding ID',
       'Vintage', 'Issuance Year', 'Days to Retirement', 'Staleness', 'Status',
       'Toucan', 'Toucan Bridger', 'Toucan Blacklisted', 'Country Code',
       'Vintaged Batch Years', 'has_ac', 'issue_retire_delta',
       'vintage_retire_delta', 'vintage_issuance_delta', 'is_blank',
       'technology', 'category'],
      dtype='object')

In [380]:
rets.category.value_counts()

REDD+             79068
RE                24963
EE                13226
HYDRO             11941
AFF/REFO           8867
EE SMALL SCALE     1319
BLUE CARBON         187
IFM                 116
Name: category, dtype: int64

In [381]:
rets.technology.value_counts()

REDD+                         78469
WIND                          21324
HYDRO                         11941
AFFORESTATION                  8103
SOLAR                          2501
EE GAS                         2172
FUEL SWITCH                    2149
BIOMASS                        1925
FUGITIVE EMISSIONS             1781
WASTE                          1408
LFG                            1314
MINING EMISSIONS REDUCTION     1190
BIOGAS                          940
REFORESTATION                   764
COOKSTOVES                      604
SOIL CARBON                     583
GEOTHERMAL                      440
SOLAR COOKERS                   425
SMALL HYDRO                     362
ROR HYDRO                       266
MANGROVE                        187
EE LIGHTING                     132
IFM                             116
LNG                             112
EE UNIVERSITY                   103
EE IRRIGATION                    76
WIND & SOLAR                     70
COGENERATION                

In [382]:
rets.to_csv("./retirements.csv", escapechar="\\", index=False)

### On-Chain Comparison

In [383]:
tokens = vcus[vcus['Toucan']]

In [384]:
tokens.head()

,Issuance Date,Sustainable Development Goals,Credit Type,Vintage Start,Vintage End,Reporting Period Start,Reporting Period End,ID,Name,Region,...,Issuance Year,Days to Retirement,Staleness,Status,Toucan,Toucan Bridger,Toucan Blacklisted,Country Code,Vintaged Batch Years,has_ac
472,2022-02-25,None,VCU,2008-01-01,2008-12-31,2008-01-01,2008-12-31,309,Bundled grid-connected wind electricity genera...,Asia,...,2022,8.0,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,1.000000,False
473,2022-02-25,None,VCU,2009-01-01,2009-09-01,2009-01-01,2009-09-01,309,Bundled grid-connected wind electricity genera...,Asia,...,2022,8.0,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,0.665753,False
679,2022-02-10,None,VCU,2011-07-01,2011-12-31,2011-07-01,2011-12-31,584,2.5 MW Rice husk based cogeneration plant at H...,Asia,...,2022,23.0,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,0.501370,False
771,2022-01-31,None,VCU,2009-01-01,2009-12-31,2009-01-01,2009-12-31,1094,Ecomapua Amazon REDD Project,Latin America,...,2022,24.0,NaN,Retired,True,https://polygonscan.com/address/0xD659a73BdBC4...,False,BRA,0.997260,True
1140,2022-01-04,None,VCU,2020-01-01,2020-12-31,2020-01-01,2020-12-31,728,CECIC Gansu Yumen Changma No.3 Wind Farm Project,Asia,...,2022,2.0,NaN,Retired,True,https://polygonscan.com/address/0x20307a574d5b...,False,CHN,1.000000,False


In [385]:
tokens = pd.merge(tokens,mapping, on="ID")

In [386]:
tokens.head()

,Issuance Date,Sustainable Development Goals,Credit Type,Vintage Start,Vintage End,Reporting Period Start,Reporting Period End,ID,Name,Region,...,Staleness,Status,Toucan,Toucan Bridger,Toucan Blacklisted,Country Code,Vintaged Batch Years,has_ac,technology,category
0,2022-02-25,None,VCU,2008-01-01,2008-12-31,2008-01-01,2008-12-31,309,Bundled grid-connected wind electricity genera...,Asia,...,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,1.000000,False,WIND,RE
1,2022-02-25,None,VCU,2009-01-01,2009-09-01,2009-01-01,2009-09-01,309,Bundled grid-connected wind electricity genera...,Asia,...,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,0.665753,False,WIND,RE
2,2022-02-10,None,VCU,2011-07-01,2011-12-31,2011-07-01,2011-12-31,584,2.5 MW Rice husk based cogeneration plant at H...,Asia,...,NaN,Retired,True,https://polygonscan.com/address/0xF36729fBfcdE...,False,IND,0.501370,False,BIOMASS,EE
3,2022-01-31,None,VCU,2009-01-01,2009-12-31,2009-01-01,2009-12-31,1094,Ecomapua Amazon REDD Project,Latin America,...,NaN,Retired,True,https://polygonscan.com/address/0xD659a73BdBC4...,False,BRA,0.997260,True,REDD+,REDD+
4,2020-12-03,None,VCU,2009-01-01,2009-12-31,2009-01-01,2009-12-31,1094,Ecomapua Amazon REDD Project,Latin America,...,NaN,Retired,True,https://polygonscan.com/address/0xD659a73BdBC4...,False,BRA,0.997260,True,REDD+,REDD+


In [387]:
tokens.groupby(['category', 'technology']).sum("Quantity")['Quantity']

category        technology                
AFF/REFO        AFFORESTATION                      772
BLUE CARBON     MANGROVE                          4200
EE              BIOGAS                           19843
                BIOMASS                         361758
                COGENERATION                     91786
                EE GAS                         2490788
                FUGITIVE EMISSIONS              877598
                INCINERATION                    274090
                LFG                             392358
                LNG                              45150
                METHANE                           6994
                MINING EMISSIONS REDUCTION        3442
                WASTE                           609742
EE SMALL SCALE  SOLAR COOKERS                    55682
HYDRO           HYDRO                         11067372
IFM             IFM                             409706
RE              ROR HYDRO                       401845
                SMALL 

In [388]:
tokens.to_csv("./tokenized_carbon.csv", escapechar="\\", index=False)